In [1]:
# dependencies
import requests
import json
from pymongo import MongoClient
from datetime import datetime
import psycopg2 # a library that allows you to connect to an existing database
from pymongo.mongo_client import MongoClient
import pymongo
import os
import sys

c:\Users\jaypa\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# set up the request parameters
params = {
  'api_key': 'E372BE0ABCFC453891D3D80F4FA7C537',
  'type': 'bestsellers',
  'amazon_domain': 'amazon.ca',
  'category_id': 'bestsellers_electronics',
  'max_page': '5'
}

In [3]:
# make the http GET request to Rainforest API
bestsellers_electronics = requests.get('https://api.rainforestapi.com/request', params)

In [4]:
# print the JSON response from Rainforest API
print(json.dumps(bestsellers_electronics.json(), indent=4))

{
    "request_info": {
        "success": true,
        "credits_used": 6,
        "credits_remaining": 94,
        "credits_used_this_request": 2
    },
    "request_parameters": {
        "type": "bestsellers",
        "amazon_domain": "amazon.ca",
        "category_id": "bestsellers_electronics",
        "max_page": "5"
    },
    "request_metadata": {
        "created_at": "2024-04-16T23:51:49.960Z",
        "processed_at": "2024-04-16T23:52:06.735Z",
        "total_time_taken": 16.77,
        "pages": [
            {
                "created_at": "2024-04-16T23:51:49.960Z",
                "processed_at": "2024-04-16T23:51:53.586Z",
                "total_time_taken": 3.63,
                "page": null,
                "amazon_url": "https://www.amazon.ca/Best-Sellers-Electronics/zgbs/electronics/ref=zg_bs_nav_0"
            },
            {
                "created_at": "2024-04-16T23:51:53.587Z",
                "processed_at": "2024-04-16T23:52:06.735Z",
                "total